1. early-stop --> epoch
2. hyper-prarm with list
- datasets * 8
- model * 3
- loss * 3
- use_ast
  
3. other hyper-prarm
- lr = 3e-5
- bsz = 8 (24g4090 + 2Bert)

4. TBC
- customized loss
- ast ckpt

In [1]:
import pandas as pd

from transformers import AdamW, get_scheduler
from tqdm import trange, tqdm

from dataset import *
from model import *
from losses import *
from metrics import *

In [ ]:
def update_metric(hist_metric, metric, l):
    hist_metric[0] += loss.item()/l
    hist_metric[1] += metric['acc'][0]/l
    hist_metric[2] += metric['acc'][1]/l
    hist_metric[3] += metric['F1'][0]/l
    hist_metric[4] += metric['F1'][1]/l
    
def precess_data(x, y, device):
    x_C = {k: v.to(device) for k, v in x[0].items()}
    x_A = {k: v.to(device) for k, v in x[1].items()}
    y = y.to(device)

In [4]:
'''
    _code_format = 'None'  -> ignore Code Snippet 
                   'Front' -> add Code BEFORE Text
                   'Back'  -> add Code BEHIND Text
                   'Separate' -> consider Code as an independent input
    _model_type =  'Multi-triage'
                   'PreTrain'
'''
def train_imm(_path, _logname, _loss_fn, _code_format='None', _model_type = 'Multi-triage', 
              _num_epochs = 20, _bsz = 8, _lr = 3e-5, 
              _ckpt = 'bert-base-uncased', _code_ckpt = 'codebert-base', 
              device = 'cuda' if torch.cuda.is_available() else 'cpu'):
    
    # DATASET Read Data: extract data
    dataset = pd.read_csv(_path)
    dataset = dataset.rename(columns={'Title_Description': 'Context', 'AST': 'AST', 'FixedByID': 'Dev', 'Name': 'Btype'})
    dataset = dataset[['Context', 'AST', 'Dev', 'Btype']]
    
    # DATASET Output: convert label to tensor
    dataset, D_ids2token, B_ids2token = label_vectorize(dataset)
    n_classes = [len(D_ids2token), len(B_ids2token)]
    # log
    logname = '../res_log/' + _logname + '.txt'
    logstr = _logname + '\\n' + '-'*60 + '\\n' + 'dataset shape:{}\nn_classes: {}'.format(dataset.shape, n_classes) + '-'*60 + '\n\n'
    print('dataset shape:{}\nn_classes: {}'.format(dataset.shape, n_classes))
    
    # DATASET Input: convert text to tensor
    dataset = text_tensorize(dataset, _ckpt, _code_format)

    # DATASET Format: split train/val/test dataset and wrap into dataloader
    train_dataloader, val_dataloader, test_dataloader = \
        split_and_wrap_dataset(dataset, _bsz)


    # MODEL load model
    if _model_type == 'Multi-triage':
        model = MetaModel(n_classes = n_classes, use_AST=(_code_format=='Separate'))
    else:
        model = PretrainModel(text_ckpt=_ckpt, code_ckpt=_code_ckpt, n_classes=n_classes, use_AST=(_code_format=='Separate'))
    model = model.to(device)

    # loss
    loss_fn = _loss_fn.to(device)

    # optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=_lr)

    # lr_scheduler
    num_epochs = _num_epochs
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps,)

    # train process
    for epoch in trange(num_epochs):
        # train
        model.train()
        train_loss = 0.0
        for x, y in train_dataloader:
            x_C, x_A, y = precess_data(x, y, device)
            outputs = model(x_C, x_A)
            loss = loss_fn(outputs, y.float())
            train_loss += loss.item()/len(train_dataloader)
            # back
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
        logstr += '{}th epoch\n train_loss: {}\n'.format(epoch, train_loss)
    
        # val
        model.eval()
        val_metric = [0.0] * 5
        for x, y in val_dataloader:
            x_C, x_A, y = precess_data(x, y, device)
            outputs = model(x_C, x_A)
            loss = loss_fn(outputs, y.float())

            metric = metrics(y, outputs, split_pos = n_classes)
            val_metric = update_metric(val_metric, metric, len(val_dataloader))
            
        logstr += '-' * 60 + '{}th epoch\n val_loss: {}\n val_acc:{}\n val_f1: {}\n'.format(epoch, val_metric[0], val_metric[1:3], val_metric[3:])

    # test
    model.eval()
    test_metric = [0.0] * 5
    for x, y in tqdm(test_dataloader):
        x_C, x_A, y = precess_data(x, y, device)
        outputs = model(x_C, x_A)
        loss = loss_fn(outputs, y.float())
        
        metric = metrics(y, outputs, split_pos = n_classes)
        test_metric = update_metric(test_metric, metric, len(test_dataloader))
        
    logstr += '-' * 60 + '\ntest_loss: {}\n test_acc:{}\n test_f1: {}'.format(test_metric[0], test_metric[1:3], test_metric[3:])
    print('test_loss: {}\n test_acc:{}\n test_f1: {}'.format(test_metric[0], test_metric[1:3], test_metric[3:]))

    with open(logname, 'w') as f:
        f.write(logstr)

In [5]:
# 8 datasets
# path, name, dataset size
pathlist = [
    ('../Data/aspnet/aspnet_1.csv', 'aspnet', 7151),
    ('../Data/efcore/efcore_1.csv', 'efcore', ),
    ('../Data/elasticSearch/elasticSearch_1.csv', 'elasticSearch'),
    ('../Data/mixedRealityToolUnity/mixedRealityToolUnity_1.csv', 'mixedRealityToolUnity'),
    ('../Data/monoGame/monoGame_1.csv', 'monoGame'),
    ('../Data/powershell/powerShell_1.csv', 'powerShell'),
    ('../Data/realmJava/realmJava_1.csv', 'realmJava'),
    ('../Data/roslyn/roslyn_1.csv', 'roslyn'),
]
losslist = [
    (nn.BCEWithLogitsLoss(), 'BCE'),
    (CustomizedBCELoss(), 'CBCE'),
    (AsymmetricLossOptimized(), 'ASL'),
]

ckptlist = [
    ('bert-base-uncased', 'Multi-triage'),  # just for tokenize
    ('bert-base-uncased', ' Bert'),
    ('roberta-base', 'Robert'),
    ('albert-base-v2', 'albert'),
]

astlist = [
    'no_AST',
    # 'use_AST',
]

In [ ]:
for path in pathlist:
    for ckpt in ckptlist:
        for loss in losslist:
            for ast in astlist:
                is_t, u_ast = (ckpt[1] == 'Multi-triage'), (ast == 'use_AST') 
                if loss[1] == 'CBCE':
                    loss[0] = CustomizedBCELoss(p_N=path[2]/2)

                logname = ' '.join([path[1], ckpt[1], loss[1], ast])
                print('-'*100, logname, '-'*100, sep='\n')
            
                train_imm(_path = path[0], _logname = '../res_log/' + logname + '.txt', 
                      _loss_fn = loss[0], _use_ast = u_ast, _is_textcnn = is_t, _ckpt = ckpt[0])

----------------------------------------------------------------------------------------------------
aspnet Multi-triage BCE no_AST
----------------------------------------------------------------------------------------------------
dataset shape:(7151, 7)
n_classes:  [61, 132]


  0%|          | 0/20 [00:00<?, ?it/s]/root/miniconda3/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1008.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 178/178 [00:00<00:00, 548.09it/s]


test_loss: 0.08315895236191459
 test_acc:[0.9914924565995686, 0.9841036927164257]
 test_f1: [0.7464875521381351, 0.0029260144551772676]
----------------------------------------------------------------------------------------------------
aspnet Multi-triage ASL no_AST
----------------------------------------------------------------------------------------------------
dataset shape:(7151, 7)
n_classes:  [61, 132]


100%|██████████| 178/178 [00:00<00:00, 491.38it/s]


test_loss: 14.192807698517706
 test_acc:[0.9911816257439314, 0.9730167064104195]
 test_f1: [0.7443590021458304, 0.3252574691984619]
----------------------------------------------------------------------------------------------------
aspnet  Bert BCE no_AST
----------------------------------------------------------------------------------------------------
dataset shape:(7151, 7)
n_classes:  [61, 132]


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 165/165 [00:05<00:00, 28.75it/s]


test_loss: 0.091461328778303
 test_acc:[0.9715756654739375, 0.9743991631450083]
 test_f1: [0.555063576446371, 0.3636511525008343]
----------------------------------------------------------------------------------------------------
efcore  Bert ASL no_AST
----------------------------------------------------------------------------------------------------
dataset shape:(6612, 7)
n_classes:  [25, 58]


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 165/165 [00:05<00:00, 28.67it/s]


test_loss: 18.672516912402536
 test_acc:[0.9661514285838961, 0.9654127500273972]
 test_f1: [0.6344624430134864, 0.4512340457911784]
----------------------------------------------------------------------------------------------------
efcore Robert BCE no_AST
----------------------------------------------------------------------------------------------------
dataset shape:(6612, 7)
n_classes:  [25, 58]


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 20%|██        | 4/20 [03:59<16:00, 60.00s/it]